In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras

from tensorflow.keras.applications import ResNet50 , VGG16, MobileNetV2
from tensorflow.keras.models import Sequential , load_model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Dropout, SpatialDropout2D, BatchNormalization, Input,Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:

# Print PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)

if cuda_available:
    # Print CUDA version
    print("CUDA version:", torch.version.cuda)
    
    # Check if cuDNN is available
    cudnn_available = torch.backends.cudnn.is_available()
    print("cuDNN available:", cudnn_available)
    
    if cudnn_available:
        # Print cuDNN version
        print("cuDNN version:", torch.backends.cudnn.version())
else:
    print("CUDA and cuDNN are not available")

PyTorch version: 2.4.0+cu118
CUDA available: True
CUDA version: 11.8
cuDNN available: True
cuDNN version: 90100


### Data Preprocessing 

In [3]:
img_height, img_width = 128, 128
batch_size=32
# Data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=45,
    width_shift_range=0.2, 
    height_shift_range=0.2,  
    shear_range=0.2, 
    zoom_range=0.3,  
    horizontal_flip=True,
    vertical_flip= True ,
    fill_mode='nearest' 
)
# No Augmentation applied for both Validation and Testing Datasets
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    'D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Dataset/Teeth_Dataset/Training',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # 'categorical' for multi-class classification, one-hot encoded vectors
)

val_generator = val_test_datagen.flow_from_directory(
    'D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Dataset/Teeth_Dataset/Validation',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    'D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Dataset/Teeth_Dataset/Testing',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [4]:


# Load the ResNet50 model
mobilenetv2_pretrained = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(img_height, img_width,3),
    pooling='avg',
    classes=7,

)

for layer in mobilenetv2_pretrained.layers:
    layer.trainable = False

# Build the model
model = Sequential()
    
model.add(mobilenetv2_pretrained)

model.add(Flatten())
# model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 2,917,447
Trainable params: 659,463
Non-trainable params: 2,257,984
________________________________________

In [5]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Define the number of steps per epoch for training, validation, and testing
train_step=train_generator.n//train_generator.batch_size
test_step=test_generator.n//test_generator.batch_size
valid_step =val_generator.n//val_generator.batch_size

# Defining early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 10,
    verbose = 0, 
    restore_best_weights = True
    )

model.fit(train_generator,
          steps_per_epoch=train_step,
          epochs=200, 
          validation_data=val_generator,
          validation_steps=valid_step,
          callbacks=[early_stopping]
            )
    

Epoch 1/200
96/96 [==============================] - 20s 171ms/step - loss: 2.0123 - accuracy: 0.2753 - val_loss: 1.3516 - val_accuracy: 0.4922
Epoch 2/200
96/96 [==============================] - 15s 157ms/step - loss: 1.5205 - accuracy: 0.4409 - val_loss: 1.1558 - val_accuracy: 0.5742
Epoch 3/200
96/96 [==============================] - 15s 155ms/step - loss: 1.3615 - accuracy: 0.5041 - val_loss: 1.0380 - val_accuracy: 0.6318
Epoch 4/200
96/96 [==============================] - 15s 154ms/step - loss: 1.2468 - accuracy: 0.5342 - val_loss: 0.9535 - val_accuracy: 0.6572
Epoch 5/200
96/96 [==============================] - 15s 152ms/step - loss: 1.2096 - accuracy: 0.5529 - val_loss: 0.9284 - val_accuracy: 0.6523
Epoch 6/200
96/96 [==============================] - 15s 155ms/step - loss: 1.1786 - accuracy: 0.5591 - val_loss: 0.9440 - val_accuracy: 0.6543
Epoch 7/200
96/96 [==============================] - 15s 153ms/step - loss: 1.1594 - accuracy: 0.5748 - val_loss: 0.8878 - val_accuracy:

In [8]:
save_model = model.save('D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Model/MobileNetV2.h5')

In [2]:
new_model = load_model('D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Model/MobileNetV2.h5')

In [13]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = new_model.evaluate(test_generator, steps=test_step)
print("Test accuracy:", test_accuracy)

32/32 [==============================] - 13s 399ms/step - loss: 0.1415 - accuracy: 0.9590
Test accuracy: 0.958984375
